In [ ]:
import cv2
import numpy as np
import pyfirmata
import threading
import time
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from filterpy.kalman import KalmanFilter
import socket

# --- Kết nối Arduino ---
port = "COM3"
board = pyfirmata.Arduino(port)
servo_pinX = board.get_pin('d:9:s')   # Servo trục X
servo_pinY = board.get_pin('d:10:s')  # Servo trục Y

# --- Khởi tạo servo ---
current_servo_x = 90
current_servo_y = 90
servo_pinX.write(current_servo_x)
servo_pinY.write(current_servo_y)
threshold_error = 10
step = 1.2

# --- Kalman Filter ---
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 1, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, 1],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0]])
kf.P *= 1000
kf.x = np.array([0, 0, 0, 0])

# --- Socket TCP ---
HOST = '127.0.0.1'  # Địa chỉ IP của máy chủ (localhost)
PORT = 65432        # Cổng để nhận dữ liệu
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen()
print("Listening for connections...")
conn, addr = server_socket.accept()
print(f"Connected by {addr}")

def receive_data():
    """Nhận dữ liệu qua socket."""
    while True:
        data = conn.recv(1024)
        if not data:
            break
        received_data = np.frombuffer(data, dtype=np.float32)
        yield received_data

# --- Load YOLO ---
model = YOLO("model.pt")

# --- DeepSORT ---
tracker = DeepSort(max_age=150, max_iou_distance=0.4, nn_budget=100)

# --- Camera ---
ws, hs = 1280, 720
cap = cv2.VideoCapture(1)
cap.set(3, ws)
cap.set(4, hs)

# --- Dữ liệu chia sẻ giữa các luồng ---
detections = []
latest_frame = None
yolo_lock = threading.Lock()

# --- Hàm YOLO chạy song song ---
def run_yolo_loop():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame_copy = latest_frame.copy()
            results = model(frame_copy)
            new_detections = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0].item()
                class_id = int(box.cls[0].item())
                if class_id == 0:  # Người
                    new_detections.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_id))
            detections = new_detections

threading.Thread(target=run_yolo_loop, daemon=True).start()

# --- Theo dõi ---
selected_feature = None

# --- Vòng lặp chính ---
for received_feature in receive_data():
    selected_feature = received_feature

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        with yolo_lock:
            latest_frame = frame.copy()

        tracks = tracker.update_tracks(detections, frame=frame)

        # Tìm đối tượng có đặc trưng gần nhất với selected_feature
        best_match = None
        min_distance = float('inf')
        for track in tracks:
            if track.is_confirmed() and track.features:
                feature = track.features[-1]
                distance = np.linalg.norm(feature - selected_feature)
                if distance < min_distance:
                    min_distance = distance
                    best_match = track

        # Nếu tìm thấy đối tượng phù hợp, điều khiển servo
        if best_match:
            x1, y1, x2, y2 = map(int, best_match.to_ltrb())
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            kf.predict()
            kf.update([cx, cy])
            predicted = kf.x[:2]
            px, py = predicted[0], predicted[1]

            # Tính sai số tâm khung hình
            center_x = ws // 2
            center_y = hs // 2
            error_x = center_x - px
            error_y = center_y - py

            # Điều chỉnh servo nếu lệch quá ngưỡng
            if abs(error_x) > threshold_error:
                current_servo_x += step if error_x > 0 else -step
            if abs(error_y) > threshold_error:
                current_servo_y += step if error_y > 0 else -step

            # Giới hạn servo trong 0–180 độ
            current_servo_x = np.clip(current_servo_x, 0, 180)
            current_servo_y = np.clip(current_servo_y, 0, 180)

            # Gửi lệnh servo
            servo_pinX.write(current_servo_x)
            servo_pinY.write(current_servo_y)

        # Hiển thị khung hình
        for track in tracks:
            if track.is_confirmed():
                x1, y1, x2, y2 = map(int, track.to_ltrb())
                color = (0, 255, 0) if track == best_match else (255, 0, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"ID {track.track_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow("Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
server_socket.close()